In [ ]:
from collections import defaultdict
import math
import pandas as pd
import re
from datetime import datetime 
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt
import pylab

print('numpy: ',np.version.version)
print('pandas: ',pd.__version__)
languages = ['en', 'es', 'eu', 'ca', 'pt', 'gl']

In [ ]:
df = pd.read_csv('../OriginalDataSet/training-tweets.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=18000)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]

In [ ]:
_df0 = df[['Language', 'Tweet']].copy()
pattern = re.compile('[ ]')
# _df0.head()

for language in languages:
    exec("%sAlphabets={}" % (language))
    exec ('{0}Size = 0'.format(language))

In [ ]:
start_time = datetime.now() 
trainDict = defaultdict(list)
for index, row in _df0.iterrows():
    sentence = ''
    tweet = row['Tweet']
    language = row['Language']
    for letter in tweet:
        if letter.isalpha() or pattern.match(letter):
            exec('if \'{let}\' not in {lang}Alphabets.keys():\n\
                     {lang}Alphabets[letter] = {lang}Size\n\
                     {lang}Size += 1'.format(let=letter, lang=language))#, language, language, language))
            sentence = sentence + letter
    trainDict[row['Language']].append(sentence)
print('Reading time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
for language in languages:
    exec('{lang}Model=np.zeros(shape=(({lang}Size+1),({lang}Size+1),({lang}Size+1)))'.format(lang=language))#, language, language, language))

#### For Bigrams

In [ ]:
# start_time = datetime.now()
# delta = 0.5
# for language, tweets in trainDict.items():
#     for tweet in tweets:
#         for i in range(len(tweet)-2):
#             first = tweet[i]
#             second = tweet[i+1]
#             exec('firstIndex = %sAlphabets[first]'%(language))
#             exec('secondIndex = %sAlphabets[second]'%(language))
#             exec('%sModel[firstIndex][secondIndex] += 1'%(language))
# for language in languages:
#     exec('%sModel = np.add(%sModel, delta)'%(language, language)) #this is where smoothing happens
#     exec('%sModelFinal = %sModel/%sModel.sum(axis=1)'%(language, language, language)) #divide all the values by the sum of the row
#     exec('%sModelFinal = np.log10(%sModelFinal)'%(language, language))
# print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
print(enAlphabets)
print('EN',enModel, enModel.shape)
# print(trainDict)

In [ ]:
# trainDict = {'en': ['this is a boy']}
# enModel[24][29][31]

#### For Trigrams

In [ ]:
start_time = datetime.now()
delta = 0.5
for language, tweets in trainDict.items():
    for tweet in tweets:
        for i in range(len(tweet)-2):
            first = tweet[i]
            second = tweet[i+1]
            third = tweet[i+2]
            exec('firstIndex = %sAlphabets[first]'%(language))
            exec('secondIndex = %sAlphabets[second]'%(language))
            exec('thirdIndex = %sAlphabets[third]'%(language))
            exec('%sModel[firstIndex][secondIndex][thirdIndex] += 1'%(language))
            
for language in languages:
    exec('for x in range({lang}Model.shape[0]):\n\
    for y in range({lang}Model.shape[0]):\n\
        sumCol = 0\n\
        for z in range({lang}Model.shape[0]):\n\
            {lang}Model[x][y][z] += delta\n\
            sumCol += {lang}Model[x][y][z]\n\
        for z in range({lang}Model.shape[0]):\n\
            {lang}Model[x][y][z] /= sumCol\n\
                \n\
{lang}Model = np.log10({lang}Model)'.format(lang=language))

print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
start_time = datetime.now()
delta = 0.5
for language, tweets in trainDict.items():
    for tweet in tweets:
        for i in range(len(tweet)-2):
            first = tweet[i]
            second = tweet[i+1]
            third = tweet[i+2]
            exec('firstIndex = %sAlphabets[first]'%(language))
            exec('secondIndex = %sAlphabets[second]'%(language))
            exec('thirdIndex = %sAlphabets[third]'%(language))
            exec('%sModel[firstIndex][secondIndex][thirdIndex] += 1'%(language))
            
for language in languages:
    exec('for x in range({lang}Model.shape[0]):\n\
    for y in range({lang}Model.shape[1]):\n\
        sumCol = 0\n\
        for z in range({lang}Model.shape[0]):\n\
            {lang}Model[x][y][z] += delta\n\
            sumCol += {lang}Model[x][y][z]\n\
        for z in range({lang}Model.shape[0]):\n\
            {lang}Model[x][y][z] /= sumCol\n\
                \n\
{lang}Model = np.log10({lang}Model)'.format(lang=language))

print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
print(enModel, file=open("enModel.txt", "a"))

## Testing the models

In [ ]:
df = pd.read_csv('../OriginalDataSet/test-tweets.txt', encoding='utf-8', error_bad_lines=False, sep='\t')#, nrows=1000)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]
_df0 = df[['TweetID', 'Language', 'Tweet']].copy()
# pattern = re.compile('[a-z ]')
# _df0.head()

In [ ]:
probability = {}
for index, row in _df0.iterrows():
    for language in languages:
        exec("%sProb=math.log10(1/6)" % (language))
    tweetID = row['TweetID']
    langTweet = row['Language']
    tweet = row['Tweet']
    
    for i in range(len(tweet)-2):
        first = tweet[i]
        second = tweet[i+1]
        third = tweet[i+2]
        for language in languages:
            exec('if ((first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys())):\n\
    prob = {lang}Model[-1][-1][-1]\n\
elif ( (first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[third]\n\
    prob = {lang}Model[-1][-1][index]\n\
elif ( (first not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[second]\n\
    prob = {lang}Model[-1][index][-1]\n\
elif ( (second not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[first]\n\
    prob = {lang}Model[index][-1][-1]\n\
elif first not in {lang}Alphabets.keys():\n\
    secondIndex = {lang}Alphabets[second]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[-1][secondIndex][thirdIndex]\n\
elif second not in {lang}Alphabets.keys():\n\
    firstIndex = {lang}Alphabets[first]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[firstIndex][-1][thirdIndex]\n\
elif third not in {lang}Alphabets.keys():\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    prob = {lang}Model[firstIndex][secondIndex][-1]\n\
else:\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[firstIndex][secondIndex][thirdIndex]\n\
{lang}Prob = prob + {lang}Prob\n'.format(lang=language))
    for langu in languages:
        exec("probability['%s'] = %sProb"%(langu, langu))
    result = max(probability, key=probability.get)
    print(tweetID, '  ', result, '  ', '%.2E' % Decimal(probability[result]), '  ', langTweet, 'correct' if (langTweet==result) else 'wrong')

# Important Stuff

In [ ]:
delta=1
B = np.arange(27.0).reshape((3,3,3))
print(B)
B[0][0][0] = 0
B[0][0][1] = 0
B[0][0][2] = 0
for x in range(B.shape[0]):
    for y in range(B.shape[0]):
        colSlice = B[x,y,:]
        colSlice = np.add(colSlice, delta)
        colSlice = colSlice/colSlice.sum()
        sumCol = 0
        for z in range(B.shape[0]):
            sumCol = sumCol + B[x][y][z]
#         print('sum for [%d][%d]'%(x,y), sumCol)
        if sumCol != 0:
            for z in range(B.shape[0]):
                B[x][y][z] /= sumCol
print(B)

In [ ]:
language = 'en'
str = 'for x in range({lang}Model.shape[0]):\n\
    for y in range({lang}Model.shape[0]):\n\
        {lang}ModelTemp = {lang}Model[x,y,:]\n\
        {lang}ModelTemp = np.add({lang}ModelTemp, delta)\n\
        {lang}ModelTemp = np.divide({lang}ModelTemp, {lang}Model.sum())\n\
        {lang}ModelTemp = np.log10({lang}ModelTemp)\n\
        {lang}Model[x,y,:] = {lang}ModelTemp\n\
{lang}Model = np.log10({lang}Model)'.format(lang=language)
print(str)

In [ ]:
lang = 'en'
str = 'if ( (not first.isalpha()) and (not second.isalpha()) and (not third.isalpha()) ):\n\
    prob = 0 \n\
elif ((first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys())):\n\
    prob = {lang}Model[-1][-1][-1]\n\
elif ( (first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[third]\n\
    prob = {lang}Model[-1][-1][index]\n\
elif ( (first not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[second]\n\
    prob = {lang}Model[-1][index][-1]\n\
elif ( (second not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[first]\n\
    prob = {lang}Model[index][-1][-1]\n\
elif first not in {lang}Alphabets.keys():\n\
    secondIndex = {lang}Alphabets[second]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[-1][secondIndex][thirdIndex]\n\
elif second not in {lang}Alphabets.keys():\n\
    firstIndex = {lang}Alphabets[first]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[firstIndex][-1][thirdIndex]\n\
elif third not in {lang}Alphabets.keys():\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    prob = {lang}Model[firstIndex][secondIndex][-1]\n\
else:\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[firstIndex][secondIndex][thirdIndex]\n\
{lang}Prob = prob + {lang}Prob\n'.format(lang='en')
    
print(str)